Goals:

- Use `numpy` to generate random samples and `scipy` to evaluate probability density functions.
- Monte Carlo integration.
- Importance sampling.



In [1]:
import numpy as np
import scipy

In [2]:
np.random.seed(101)
N = 100
pi = np.random.beta(1, 1, size=1)
y = np.random.binomial(n = 1, p = pi, size = N)

#print(pi)
print(y)
sy = y.sum()
print(sy)

[0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
12


We will sample from the prior $\pi_k \sim Beta(1, 1)$.

$$p(y) = \int p(y | \pi) p(\pi) d\pi = \int \prod_n p(y_n | \pi) p(\pi) d\pi$$.

Note that $\prod_n p(y_n | \pi) = \prod_n \pi^{y_n} (1 - \pi)^{1-y_n} = \pi^{s_y} (1 - \pi)^{N - s_y}$, where $s_y = \sum_n y_n$.

$$\int \prod_n p(y_n | \pi) p(\pi) d\pi = \int \pi^{s_y} (1 - \pi)^{N-s_y} p(\pi) d\pi \approx \frac{1}{K} \sum_k \pi_k^{s_y} (1 - \pi_k)^{N-s_y}$$

In [5]:
def log_pmf(sy, N, pi):
  return sy * np.log(pi) + (N - sy) * np.log(1 - pi)

def pmf(sy, N, pi):
  return pi**sy * (1 - pi)**(N - sy)


In [20]:
sample_size = 100000
pi_samples = np.random.beta(1, 1, size=sample_size)

a = []
for pi_sample in pi_samples:
    a.append(pmf(sy, N, pi_sample))
pmf_values = np.array(a)

In [21]:
pmf_values = np.array([pmf(sy, N, pi_sample) for pi_sample in pi_samples])
marginal_mc_estimate1 = pmf_values.mean()
print(marginal_mc_estimate1)



9.36929646578975e-18


In [22]:
# The truth is B(a, b).
a0 = 1
a = sy + a0
b0 = 1
b = N - sy + b0

In [23]:
scipy.special.beta(a, b)

9.425734650479863e-18

What is the true marginal likelihood?

Bernoulli-Beta conjugacy implies that the posterior is also in the same family as the prior.

$$p(\pi | y) = \frac{p(y | \pi) p(\pi)}{p(y)} \Rightarrow p(y) = \frac{p(y|\pi) p(\pi)}{p(\pi|y)}$$

This is true for any $\pi$, so we can evaluate the marginal likelihood.



In [ ]:
some_pi = 0.5
likelihood = pmf(sy, N, some_pi)
prior = scipy.stats.beta.pdf(some_pi, 1, 1)
posterior = scipy.stats.beta.pdf(some_pi, sy + 1, N - sy + 1)
true_marginal_likelihood = likelihood * prior / posterior
print(true_marginal_likelihood)

log_likelihood = log_pmf(sy, N, some_pi)
log_prior = scipy.stats.beta.logpdf(some_pi, 1, 1)
log_posterior = scipy.stats.beta.logpdf(some_pi, sy + 1, N - sy + 1)

true_log_marginal_likelihood = log_likelihood + log_prior - log_posterior
print(true_log_marginal_likelihood)

9.425734650479846e-18
-39.20308799659594


In [24]:

log_pmf_values = np.array([log_pmf(sy, N, pi_sample) for pi_sample in pi_samples])
log_sum_vals = np.logaddexp.reduce(log_pmf_values)
marginal_mc_estimate2 = np.exp(log_sum_vals) / sample_size
print(marginal_mc_estimate2)


9.369296465785894e-18


In [25]:

log_marginal_likelihood_estimate = log_sum_vals - np.log(sample_size)
print(log_marginal_likelihood_estimate)

-39.20909366415555


## Importance sampling

Estimating the tail probability.

Let $X \sim Normal(0, 1)$. Estimate $P(X > x_0)$ for some extreme $x_0$.

In [26]:
# Naive estimate:
x0 = 5
estimate1 = 1 - scipy.stats.norm.cdf(x0, 0, 1)
print(estimate1)


2.866515719235352e-07


What we want is $P(X > x_0) = \mathbb{E}[1[X > x_0]] = \int 1[x > x_0] \phi(x) dx$ where $\phi(x)$ is normal pdf.

Naive Monte Carlo integration will not work well since it will barely draw any sample where $x > x_0$ for extreme $x_0$.

In [27]:
x = np.random.normal(size=10000000)
estimate2 = np.mean(x > x0)
print(estimate2)

4e-07


Let's formulate importance sampling estimator.

$\int 1[x > x_0] \phi(x) = \int 1[x > x_0] \frac{\phi(x)}{f(x)} f(x) dx$ for some importance distribution $f$. We define $w(x) = \phi(x)/f(x)$.

We will choose $f$ so that it has mass concentrated around $x_0$.

The IS estimator is given by $P(X > x_0) \approx \frac{1}{K} \sum_k w(x_k) 1[x_k > x_0]$.

In [28]:
x = np.random.normal(size=10000, loc=x0, scale=1)
weights = scipy.stats.norm.pdf(x, 0, 1) / scipy.stats.norm.pdf(x, x0, 1)
estimate3 = np.mean(weights * (x > x0))
print(estimate3)

2.790515826943391e-07


What happens if we choose a different importance distribution?

In [29]:
x = np.random.normal(size=100000, loc=x0, scale=2)
weights = scipy.stats.norm.pdf(x, 0, 1) / scipy.stats.norm.pdf(x, x0, 1)
print(np.mean(weights * (x > x0)))


1.4950627747838689e-07


In [ ]:
x = x0 + np.random.exponential(size=10000, scale=1)
weights = scipy.stats.norm.pdf(x, 0, 1) / (scipy.stats.expon.pdf(x, scale=1) * np.exp(x0))
print(np.mean(weights * (x > x0)))


0.0013270959461802764
